# Phase 8 & 9: Model Evaluation and Interpretation

This notebook combines Phase 8 (Model Evaluation) and Phase 9 (Model Interpretation) into a comprehensive analysis:

## Phase 8: Model Evaluation
- Confusion matrices for all models
- ROC curves and AUC comparison
- Precision-Recall curves
- Calibration curves (reliability diagrams)
- Error analysis (what each model gets wrong)
- Subgroup analysis (age, gender, BMI, race)
- With-labs vs without-labs comparison
- Regression evaluation (residuals, predicted vs actual)

## Phase 9: Model Interpretation
- SHAP values for best model (LightGBM)
- Global feature importance
- SHAP dependence plots
- Modifiable vs non-modifiable risk factors
- Actionable insights

In [ ]:
# Standard imports
import json
import sys
import warnings
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

# ML imports
import joblib
import shap
from sklearn.metrics import (
    accuracy_score, f1_score, roc_auc_score, precision_score, recall_score,
    confusion_matrix, classification_report,
    mean_squared_error, mean_absolute_error, r2_score
)
from sklearn.inspection import permutation_importance

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=UserWarning)

# Add project root to path
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root))

# Project imports
from src.models.evaluate import (
    compute_classification_metrics, compute_regression_metrics,
    plot_confusion_matrix, plot_roc_curves, plot_precision_recall_curves,
    plot_calibration_curve, plot_residuals, plot_predicted_vs_actual,
    compare_models_table, DIABETES_LABELS, DIABETES_COLORS
)

# Set style
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams.update({
    'figure.dpi': 100,
    'savefig.dpi': 300,
    'font.size': 11,
    'axes.titlesize': 12,
    'axes.labelsize': 11,
})

print(f"Project root: {project_root}")
print(f"SHAP version: {shap.__version__}")

In [ ]:
# Configuration
RANDOM_STATE = 42

# Paths
DATA_DIR = project_root / 'data' / 'processed'
INTERIM_DIR = project_root / 'data' / 'interim'
MODELS_DIR = project_root / 'models' / 'advanced'
FIGURES_DIR = project_root / 'reports' / 'figures'

# Create figures directory
FIGURES_DIR.mkdir(parents=True, exist_ok=True)

print(f"Data directory: {DATA_DIR}")
print(f"Models directory: {MODELS_DIR}")
print(f"Figures directory: {FIGURES_DIR}")

## 1. Load Data and Models

In [ ]:
# Load feature matrices
print("Loading datasets...")

X_with_labs_minimal = pd.read_parquet(DATA_DIR / 'X_with_labs_minimal.parquet')
X_with_labs_full = pd.read_parquet(DATA_DIR / 'X_with_labs_full.parquet')
X_without_labs_minimal = pd.read_parquet(DATA_DIR / 'X_without_labs_minimal.parquet')
X_without_labs_full = pd.read_parquet(DATA_DIR / 'X_without_labs_full.parquet')

# Classification target
y_classification = pd.read_parquet(DATA_DIR / 'y_with_labs_minimal.parquet')['DIABETES_STATUS']

# Regression target (HbA1c)
study_pop = pd.read_parquet(INTERIM_DIR / 'study_population.parquet')
y_regression = study_pop.loc[y_classification.index, 'LBXGH']

print(f"\nDataset shapes:")
print(f"  X_with_labs_minimal:    {X_with_labs_minimal.shape}")
print(f"  X_with_labs_full:       {X_with_labs_full.shape}")
print(f"  X_without_labs_minimal: {X_without_labs_minimal.shape}")
print(f"  X_without_labs_full:    {X_without_labs_full.shape}")
print(f"  y_classification:       {y_classification.shape}")
print(f"  y_regression:           {y_regression.shape} (non-null: {y_regression.notna().sum()})")

In [ ]:
# Load demographic data for subgroup analysis
demographics = study_pop.loc[y_classification.index, ['RIDAGEYR', 'RIAGENDR', 'RIDRETH3']].copy()
demographics['BMXBMI'] = study_pop.loc[y_classification.index, 'BMXBMI']

# Create age groups
demographics['AGE_GROUP'] = pd.cut(
    demographics['RIDAGEYR'],
    bins=[18, 40, 60, 100],
    labels=['Young (18-39)', 'Middle (40-59)', 'Older (60+)']
)

# Create BMI categories
demographics['BMI_CAT'] = pd.cut(
    demographics['BMXBMI'],
    bins=[0, 25, 30, 100],
    labels=['Normal (<25)', 'Overweight (25-30)', 'Obese (30+)']
)

# Gender labels
demographics['GENDER'] = demographics['RIAGENDR'].map({1: 'Male', 2: 'Female'})

# Race/ethnicity labels (RIDRETH3)
race_map = {
    1: 'Mexican American',
    2: 'Other Hispanic',
    3: 'Non-Hispanic White',
    4: 'Non-Hispanic Black',
    6: 'Non-Hispanic Asian',
    7: 'Other/Multi-Racial'
}
demographics['RACE'] = demographics['RIDRETH3'].map(race_map)

print("Demographics loaded:")
print(demographics[['AGE_GROUP', 'GENDER', 'BMI_CAT', 'RACE']].describe())

In [ ]:
# Load trained models
print("Loading models...")

# Classification models
lgb_cls_with_labs = joblib.load(MODELS_DIR / 'classification' / 'lgb_with_labs.joblib')
lgb_cls_without_labs = joblib.load(MODELS_DIR / 'classification' / 'lgb_without_labs.joblib')

mlp_cls_with_labs_data = joblib.load(MODELS_DIR / 'classification' / 'mlp_with_labs.joblib')
mlp_cls_without_labs_data = joblib.load(MODELS_DIR / 'classification' / 'mlp_without_labs.joblib')

# MLP models come with scalers
if isinstance(mlp_cls_with_labs_data, tuple):
    mlp_cls_with_labs, scaler_cls_with = mlp_cls_with_labs_data
else:
    mlp_cls_with_labs = mlp_cls_with_labs_data
    scaler_cls_with = None

if isinstance(mlp_cls_without_labs_data, tuple):
    mlp_cls_without_labs, scaler_cls_without = mlp_cls_without_labs_data
else:
    mlp_cls_without_labs = mlp_cls_without_labs_data
    scaler_cls_without = None

# Regression models
lgb_reg_with_labs = joblib.load(MODELS_DIR / 'regression' / 'lgb_with_labs.joblib')
lgb_reg_without_labs = joblib.load(MODELS_DIR / 'regression' / 'lgb_without_labs.joblib')

mlp_reg_with_labs_data = joblib.load(MODELS_DIR / 'regression' / 'mlp_with_labs.joblib')
mlp_reg_without_labs_data = joblib.load(MODELS_DIR / 'regression' / 'mlp_without_labs.joblib')

if isinstance(mlp_reg_with_labs_data, tuple):
    mlp_reg_with_labs, scaler_reg_with = mlp_reg_with_labs_data
else:
    mlp_reg_with_labs = mlp_reg_with_labs_data
    scaler_reg_with = None

if isinstance(mlp_reg_without_labs_data, tuple):
    mlp_reg_without_labs, scaler_reg_without = mlp_reg_without_labs_data
else:
    mlp_reg_without_labs = mlp_reg_without_labs_data
    scaler_reg_without = None

print("\nModels loaded:")
print("  Classification: LightGBM (with/without labs), MLP (with/without labs)")
print("  Regression: LightGBM (with/without labs), MLP (with/without labs)")

In [ ]:
# Try to load PyTorch model if available
pytorch_model = None
pytorch_scaler = None
pytorch_path = MODELS_DIR / 'classification' / 'pytorch_with_labs.pt'

if pytorch_path.exists():
    try:
        import torch
        import torch.nn as nn
        
        # Load the checkpoint
        checkpoint = torch.load(pytorch_path, weights_only=False)
        
        # Define the model architecture (must match training)
        class DiabetesClassifier(nn.Module):
            def __init__(self, n_features, n_classes=3, dropout_rate=0.3):
                super().__init__()
                self.layer1 = nn.Sequential(
                    nn.Linear(n_features, 128),
                    nn.BatchNorm1d(128),
                    nn.ReLU(),
                    nn.Dropout(dropout_rate)
                )
                self.layer2 = nn.Sequential(
                    nn.Linear(128, 64),
                    nn.BatchNorm1d(64),
                    nn.ReLU(),
                    nn.Dropout(dropout_rate)
                )
                self.layer3 = nn.Sequential(
                    nn.Linear(64, 32),
                    nn.BatchNorm1d(32),
                    nn.ReLU(),
                    nn.Dropout(dropout_rate)
                )
                self.output = nn.Linear(32, n_classes)
            
            def forward(self, x):
                x = self.layer1(x)
                x = self.layer2(x)
                x = self.layer3(x)
                return self.output(x)
        
        # Get model parameters from checkpoint
        n_features = checkpoint['n_features']
        n_classes = checkpoint['n_classes']
        dropout_rate = checkpoint['dropout_rate']
        
        # Create model and load state dict
        pytorch_model = DiabetesClassifier(n_features, n_classes, dropout_rate)
        pytorch_model.load_state_dict(checkpoint['model_state_dict'])
        pytorch_model.eval()
        
        # Also get the scaler
        pytorch_scaler = checkpoint.get('scaler')
        
        print(f"PyTorch model loaded from {pytorch_path}")
        print(f"  n_features: {n_features}, n_classes: {n_classes}")
        print(f"  Saved metrics: {checkpoint.get('metrics', {})}")
    except Exception as e:
        print(f"Could not load PyTorch model: {e}")
        pytorch_model = None
else:
    print("PyTorch model not found - will evaluate sklearn models only")

In [ ]:
# Create train/val/test splits (same as training)
from sklearn.model_selection import train_test_split

# First split: train+val (85%) and test (15%)
train_val_idx, test_idx = train_test_split(
    y_classification.index,
    test_size=0.15,
    random_state=RANDOM_STATE,
    stratify=y_classification
)

# Second split: train (70%) and val (15%) from train+val
train_idx, val_idx = train_test_split(
    train_val_idx,
    test_size=0.15/0.85,  # 15% of original = 15/85 of train+val
    random_state=RANDOM_STATE,
    stratify=y_classification.loc[train_val_idx]
)

print(f"Data splits:")
print(f"  Train: {len(train_idx):,} ({len(train_idx)/len(y_classification)*100:.1f}%)")
print(f"  Val:   {len(val_idx):,} ({len(val_idx)/len(y_classification)*100:.1f}%)")
print(f"  Test:  {len(test_idx):,} ({len(test_idx)/len(y_classification)*100:.1f}%)")

In [ ]:
# Create test sets for each model type
def get_test_data(X, y, test_idx):
    """Get test data with matching indices."""
    common_idx = test_idx.intersection(y.index)
    return X.loc[common_idx], y.loc[common_idx]

# Classification test sets
X_test_lgb_with, y_test_cls = get_test_data(X_with_labs_minimal, y_classification, test_idx)
X_test_lgb_without, _ = get_test_data(X_without_labs_minimal, y_classification, test_idx)
X_test_mlp_with, _ = get_test_data(X_with_labs_full, y_classification, test_idx)
X_test_mlp_without, _ = get_test_data(X_without_labs_full, y_classification, test_idx)

# Regression test sets
y_reg_valid = y_regression.dropna()
X_test_lgb_with_reg, y_test_reg = get_test_data(X_with_labs_minimal, y_reg_valid, test_idx)
X_test_lgb_without_reg, _ = get_test_data(X_without_labs_minimal, y_reg_valid, test_idx)
X_test_mlp_with_reg, _ = get_test_data(X_with_labs_full, y_reg_valid, test_idx)
X_test_mlp_without_reg, _ = get_test_data(X_without_labs_full, y_reg_valid, test_idx)

print(f"Classification test set: {len(y_test_cls)} samples")
print(f"Regression test set: {len(y_test_reg)} samples (with valid HbA1c)")

---
# Part 1: Classification Evaluation (Phase 8)

## 2. Generate Predictions

In [ ]:
# Generate predictions for all classification models
predictions = {}
probabilities = {}

# LightGBM with labs
predictions['LightGBM (with labs)'] = lgb_cls_with_labs.predict(X_test_lgb_with.values)
probabilities['LightGBM (with labs)'] = lgb_cls_with_labs.predict_proba(X_test_lgb_with.values)

# LightGBM without labs
predictions['LightGBM (without labs)'] = lgb_cls_without_labs.predict(X_test_lgb_without.values)
probabilities['LightGBM (without labs)'] = lgb_cls_without_labs.predict_proba(X_test_lgb_without.values)

# MLP with labs
if scaler_cls_with is not None:
    X_test_scaled = scaler_cls_with.transform(X_test_mlp_with)
else:
    X_test_scaled = X_test_mlp_with.values
predictions['MLP (with labs)'] = mlp_cls_with_labs.predict(X_test_scaled)
probabilities['MLP (with labs)'] = mlp_cls_with_labs.predict_proba(X_test_scaled)

# MLP without labs
if scaler_cls_without is not None:
    X_test_scaled = scaler_cls_without.transform(X_test_mlp_without)
else:
    X_test_scaled = X_test_mlp_without.values
predictions['MLP (without labs)'] = mlp_cls_without_labs.predict(X_test_scaled)
probabilities['MLP (without labs)'] = mlp_cls_without_labs.predict_proba(X_test_scaled)

# PyTorch if available
if pytorch_model is not None:
    import torch
    import torch.nn.functional as F
    
    # Use the saved scaler from the checkpoint
    if pytorch_scaler is not None:
        X_test_pytorch_scaled = pytorch_scaler.transform(X_test_mlp_with)
    else:
        X_test_pytorch_scaled = X_test_mlp_with.values
    
    with torch.no_grad():
        X_tensor = torch.FloatTensor(X_test_pytorch_scaled)
        logits = pytorch_model(X_tensor)
        probs = F.softmax(logits, dim=1).numpy()
        preds = np.argmax(probs, axis=1)
    
    predictions['PyTorch (with labs)'] = preds
    probabilities['PyTorch (with labs)'] = probs

print(f"Generated predictions for {len(predictions)} models")

In [ ]:
# Compute metrics for all models
results = {}
for model_name in predictions:
    y_pred = predictions[model_name]
    y_prob = probabilities[model_name]
    results[model_name] = compute_classification_metrics(y_test_cls.values, y_pred, y_prob)

# Display comparison table
metrics_df = compare_models_table(
    results,
    metrics=['accuracy', 'f1_macro', 'roc_auc_ovr', 'precision_macro', 'recall_macro'],
    sort_by='f1_macro',
    ascending=False
)

print("\n" + "="*70)
print("CLASSIFICATION MODEL COMPARISON (Test Set)")
print("="*70)
print(metrics_df.to_string())

## 3. Confusion Matrices

In [ ]:
# Plot confusion matrices for all models
n_models = len(predictions)
n_cols = 2
n_rows = (n_models + 1) // 2

fig, axes = plt.subplots(n_rows, n_cols, figsize=(12, 5*n_rows))
axes = axes.flatten()

for i, (model_name, y_pred) in enumerate(predictions.items()):
    plot_confusion_matrix(
        y_test_cls.values, y_pred,
        ax=axes[i],
        normalize=True,
        title=f'{model_name}\nF1={results[model_name]["f1_macro"]:.3f}'
    )

# Hide empty subplots
for j in range(i+1, len(axes)):
    axes[j].set_visible(False)

plt.suptitle('Confusion Matrices (Normalized) - Test Set', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
fig.savefig(FIGURES_DIR / 'phase8_confusion_matrices.png', dpi=300, bbox_inches='tight')
plt.show()

## 4. ROC Curves Comparison

In [ ]:
# Plot ROC curves for all models (one plot per class)
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

class_names = {0: 'No Diabetes', 1: 'Prediabetes', 2: 'Diabetes'}
colors = plt.cm.tab10(np.linspace(0, 1, len(probabilities)))

for class_idx, class_name in class_names.items():
    ax = axes[class_idx]
    
    for (model_name, y_prob), color in zip(probabilities.items(), colors):
        y_true_binary = (y_test_cls.values == class_idx).astype(int)
        
        from sklearn.metrics import roc_curve, auc
        fpr, tpr, _ = roc_curve(y_true_binary, y_prob[:, class_idx])
        roc_auc = auc(fpr, tpr)
        
        # Shorten model name for legend
        short_name = model_name.replace(' (with labs)', '+labs').replace(' (without labs)', '-labs')
        ax.plot(fpr, tpr, color=color, linewidth=2, label=f'{short_name} ({roc_auc:.3f})')
    
    ax.plot([0, 1], [0, 1], 'k--', alpha=0.5)
    ax.set_xlabel('False Positive Rate')
    ax.set_ylabel('True Positive Rate')
    ax.set_title(f'ROC Curve: {class_name}')
    ax.legend(loc='lower right', fontsize=9)
    ax.grid(True, alpha=0.3)

plt.suptitle('ROC Curves by Class (One-vs-Rest)', fontsize=14, fontweight='bold')
plt.tight_layout()
fig.savefig(FIGURES_DIR / 'phase8_roc_curves_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

## 5. Precision-Recall Curves

In [ ]:
# Precision-Recall curves
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

for class_idx, class_name in class_names.items():
    ax = axes[class_idx]
    
    # Calculate prevalence for this class
    prevalence = (y_test_cls.values == class_idx).mean()
    
    for (model_name, y_prob), color in zip(probabilities.items(), colors):
        y_true_binary = (y_test_cls.values == class_idx).astype(int)
        
        from sklearn.metrics import precision_recall_curve, average_precision_score
        precision, recall, _ = precision_recall_curve(y_true_binary, y_prob[:, class_idx])
        ap = average_precision_score(y_true_binary, y_prob[:, class_idx])
        
        short_name = model_name.replace(' (with labs)', '+labs').replace(' (without labs)', '-labs')
        ax.plot(recall, precision, color=color, linewidth=2, label=f'{short_name} (AP={ap:.3f})')
    
    # Baseline (prevalence)
    ax.axhline(y=prevalence, color='gray', linestyle='--', alpha=0.5, label=f'Baseline ({prevalence:.2f})')
    
    ax.set_xlabel('Recall')
    ax.set_ylabel('Precision')
    ax.set_title(f'PR Curve: {class_name}')
    ax.legend(loc='upper right', fontsize=8)
    ax.grid(True, alpha=0.3)
    ax.set_xlim([0, 1])
    ax.set_ylim([0, 1])

plt.suptitle('Precision-Recall Curves by Class', fontsize=14, fontweight='bold')
plt.tight_layout()
fig.savefig(FIGURES_DIR / 'phase8_pr_curves_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

## 6. Calibration Curves

In [ ]:
# Calibration curves for best model (LightGBM with labs) for all classes
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

best_model = 'LightGBM (with labs)'
y_prob = probabilities[best_model]

from sklearn.calibration import calibration_curve

for class_idx, class_name in class_names.items():
    ax = axes[class_idx]
    
    y_true_binary = (y_test_cls.values == class_idx).astype(int)
    prob_class = y_prob[:, class_idx]
    
    # Calibration curve
    prob_true, prob_pred = calibration_curve(y_true_binary, prob_class, n_bins=10)
    
    ax.plot(prob_pred, prob_true, 's-', color=DIABETES_COLORS[class_idx], 
            linewidth=2, markersize=8, label=f'{best_model}')
    ax.plot([0, 1], [0, 1], 'k--', alpha=0.5, label='Perfectly Calibrated')
    
    # Add histogram of predictions
    ax2 = ax.twinx()
    ax2.hist(prob_class, bins=10, alpha=0.3, color=DIABETES_COLORS[class_idx])
    ax2.set_ylabel('Count', alpha=0.5)
    
    ax.set_xlabel('Mean Predicted Probability')
    ax.set_ylabel('Fraction of Positives')
    ax.set_title(f'Calibration: {class_name}')
    ax.legend(loc='upper left')
    ax.grid(True, alpha=0.3)
    ax.set_xlim([0, 1])
    ax.set_ylim([0, 1])

plt.suptitle(f'Calibration Curves - {best_model}', fontsize=14, fontweight='bold')
plt.tight_layout()
fig.savefig(FIGURES_DIR / 'phase8_calibration_curves.png', dpi=300, bbox_inches='tight')
plt.show()

## 7. Error Analysis

In [ ]:
# Analyze errors for best model
best_model = 'LightGBM (with labs)'
y_pred = predictions[best_model]
y_prob = probabilities[best_model]

# Create error analysis dataframe
error_df = pd.DataFrame({
    'y_true': y_test_cls.values,
    'y_pred': y_pred,
    'correct': y_test_cls.values == y_pred,
    'prob_0': y_prob[:, 0],
    'prob_1': y_prob[:, 1],
    'prob_2': y_prob[:, 2],
    'max_prob': y_prob.max(axis=1),
}, index=y_test_cls.index)

# Add demographics
error_df = error_df.join(demographics.loc[error_df.index])

# Error rate by true class
print("\nError Rate by True Class:")
print("="*40)
for cls in [0, 1, 2]:
    mask = error_df['y_true'] == cls
    error_rate = 1 - error_df.loc[mask, 'correct'].mean()
    n = mask.sum()
    print(f"  {DIABETES_LABELS[cls]}: {error_rate:.1%} ({int(n * error_rate)}/{n} errors)")

In [ ]:
# What gets misclassified most often?
print("\nMisclassification Patterns:")
print("="*50)

errors = error_df[~error_df['correct']].copy()

# Cross-tabulate true vs predicted for errors
error_patterns = pd.crosstab(
    errors['y_true'].map(DIABETES_LABELS),
    errors['y_pred'].map(DIABETES_LABELS),
    margins=True
)
print(error_patterns)

print(f"\nTotal errors: {len(errors)} ({len(errors)/len(error_df)*100:.1f}%)")

In [ ]:
# Confidence in errors vs correct predictions
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Histogram of max probability for correct vs incorrect
ax = axes[0]
ax.hist(error_df.loc[error_df['correct'], 'max_prob'], bins=20, alpha=0.7, 
        label=f'Correct (n={error_df["correct"].sum()})', color='green')
ax.hist(error_df.loc[~error_df['correct'], 'max_prob'], bins=20, alpha=0.7,
        label=f'Incorrect (n={(~error_df["correct"]).sum()})', color='red')
ax.set_xlabel('Max Predicted Probability (Confidence)')
ax.set_ylabel('Count')
ax.set_title('Prediction Confidence: Correct vs Incorrect')
ax.legend()
ax.grid(True, alpha=0.3)

# Error rate by confidence bin
ax = axes[1]
error_df['confidence_bin'] = pd.cut(error_df['max_prob'], bins=[0, 0.4, 0.5, 0.6, 0.7, 0.8, 1.0])
error_by_conf = error_df.groupby('confidence_bin', observed=True)['correct'].agg(['mean', 'count'])
error_by_conf['error_rate'] = 1 - error_by_conf['mean']

x = range(len(error_by_conf))
ax.bar(x, error_by_conf['error_rate'], color='coral', alpha=0.7)
ax.set_xticks(x)
ax.set_xticklabels([str(i) for i in error_by_conf.index], rotation=45, ha='right')
ax.set_xlabel('Confidence Bin')
ax.set_ylabel('Error Rate')
ax.set_title('Error Rate by Prediction Confidence')

# Add count labels
for i, (idx, row) in enumerate(error_by_conf.iterrows()):
    ax.annotate(f'n={int(row["count"])}', xy=(i, row['error_rate']), 
                xytext=(0, 5), textcoords='offset points', ha='center', fontsize=9)

ax.grid(True, alpha=0.3, axis='y')

plt.suptitle(f'Error Analysis - {best_model}', fontsize=14, fontweight='bold')
plt.tight_layout()
fig.savefig(FIGURES_DIR / 'phase8_error_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

## 8. Subgroup Analysis

In [ ]:
def evaluate_subgroup(df, subgroup_col, model_name='LightGBM (with labs)'):
    """Evaluate model performance by subgroup."""
    y_pred = predictions[model_name]
    y_prob = probabilities[model_name]
    
    results = []
    for group in df[subgroup_col].dropna().unique():
        mask = df[subgroup_col] == group
        if mask.sum() < 50:  # Skip small groups
            continue
        
        y_true_group = y_test_cls.values[mask]
        y_pred_group = y_pred[mask]
        y_prob_group = y_prob[mask]
        
        metrics = compute_classification_metrics(y_true_group, y_pred_group, y_prob_group)
        results.append({
            'Subgroup': group,
            'N': mask.sum(),
            'Accuracy': metrics['accuracy'],
            'F1 Macro': metrics['f1_macro'],
            'ROC AUC': metrics.get('roc_auc_ovr', np.nan)
        })
    
    return pd.DataFrame(results).sort_values('Subgroup')

# Match error_df index
subgroup_df = error_df.copy()

In [ ]:
# Performance by Age Group
print("\nPerformance by Age Group:")
print("="*60)
age_results = evaluate_subgroup(subgroup_df, 'AGE_GROUP')
print(age_results.to_string(index=False))

In [ ]:
# Performance by Gender
print("\nPerformance by Gender:")
print("="*60)
gender_results = evaluate_subgroup(subgroup_df, 'GENDER')
print(gender_results.to_string(index=False))

In [ ]:
# Performance by BMI Category
print("\nPerformance by BMI Category:")
print("="*60)
bmi_results = evaluate_subgroup(subgroup_df, 'BMI_CAT')
print(bmi_results.to_string(index=False))

In [ ]:
# Performance by Race/Ethnicity
print("\nPerformance by Race/Ethnicity:")
print("="*60)
race_results = evaluate_subgroup(subgroup_df, 'RACE')
print(race_results.to_string(index=False))

In [ ]:
# Visualize subgroup performance
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Age
ax = axes[0, 0]
if len(age_results) > 0:
    x = range(len(age_results))
    ax.bar(x, age_results['F1 Macro'], color='steelblue', alpha=0.7)
    ax.set_xticks(x)
    ax.set_xticklabels(age_results['Subgroup'], rotation=45, ha='right')
    ax.set_ylabel('F1 Macro')
    ax.set_title('Performance by Age Group')
    ax.set_ylim([0, 0.8])
    for i, (_, row) in enumerate(age_results.iterrows()):
        ax.annotate(f'{row["F1 Macro"]:.3f}\n(n={row["N"]})', xy=(i, row['F1 Macro']+0.02), ha='center', fontsize=9)
ax.grid(True, alpha=0.3, axis='y')

# Gender
ax = axes[0, 1]
if len(gender_results) > 0:
    x = range(len(gender_results))
    ax.bar(x, gender_results['F1 Macro'], color='coral', alpha=0.7)
    ax.set_xticks(x)
    ax.set_xticklabels(gender_results['Subgroup'])
    ax.set_ylabel('F1 Macro')
    ax.set_title('Performance by Gender')
    ax.set_ylim([0, 0.8])
    for i, (_, row) in enumerate(gender_results.iterrows()):
        ax.annotate(f'{row["F1 Macro"]:.3f}\n(n={row["N"]})', xy=(i, row['F1 Macro']+0.02), ha='center', fontsize=9)
ax.grid(True, alpha=0.3, axis='y')

# BMI
ax = axes[1, 0]
if len(bmi_results) > 0:
    x = range(len(bmi_results))
    ax.bar(x, bmi_results['F1 Macro'], color='green', alpha=0.7)
    ax.set_xticks(x)
    ax.set_xticklabels(bmi_results['Subgroup'], rotation=45, ha='right')
    ax.set_ylabel('F1 Macro')
    ax.set_title('Performance by BMI Category')
    ax.set_ylim([0, 0.8])
    for i, (_, row) in enumerate(bmi_results.iterrows()):
        ax.annotate(f'{row["F1 Macro"]:.3f}\n(n={row["N"]})', xy=(i, row['F1 Macro']+0.02), ha='center', fontsize=9)
ax.grid(True, alpha=0.3, axis='y')

# Race
ax = axes[1, 1]
if len(race_results) > 0:
    x = range(len(race_results))
    ax.bar(x, race_results['F1 Macro'], color='purple', alpha=0.7)
    ax.set_xticks(x)
    ax.set_xticklabels(race_results['Subgroup'], rotation=45, ha='right')
    ax.set_ylabel('F1 Macro')
    ax.set_title('Performance by Race/Ethnicity')
    ax.set_ylim([0, 0.8])
    for i, (_, row) in enumerate(race_results.iterrows()):
        ax.annotate(f'{row["F1 Macro"]:.3f}', xy=(i, row['F1 Macro']+0.02), ha='center', fontsize=8, rotation=90)
ax.grid(True, alpha=0.3, axis='y')

plt.suptitle('Model Performance by Subgroup - LightGBM (with labs)', fontsize=14, fontweight='bold')
plt.tight_layout()
fig.savefig(FIGURES_DIR / 'phase8_subgroup_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

## 9. With-Labs vs Without-Labs Comparison

In [ ]:
# Compare with-labs vs without-labs performance
comparison_data = []

for model_type in ['LightGBM', 'MLP']:
    with_labs_key = f'{model_type} (with labs)'
    without_labs_key = f'{model_type} (without labs)'
    
    if with_labs_key in results and without_labs_key in results:
        with_labs_f1 = results[with_labs_key]['f1_macro']
        without_labs_f1 = results[without_labs_key]['f1_macro']
        with_labs_auc = results[with_labs_key]['roc_auc_ovr']
        without_labs_auc = results[without_labs_key]['roc_auc_ovr']
        
        comparison_data.append({
            'Model': model_type,
            'F1 (with labs)': with_labs_f1,
            'F1 (without labs)': without_labs_f1,
            'F1 Drop': with_labs_f1 - without_labs_f1,
            'F1 Drop %': (with_labs_f1 - without_labs_f1) / with_labs_f1 * 100,
            'AUC (with labs)': with_labs_auc,
            'AUC (without labs)': without_labs_auc,
            'AUC Drop': with_labs_auc - without_labs_auc,
        })

comparison_df = pd.DataFrame(comparison_data)

print("\nWith-Labs vs Without-Labs Comparison:")
print("="*80)
print(comparison_df.round(4).to_string(index=False))

In [ ]:
# Visualize the comparison
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# F1 comparison
ax = axes[0]
x = np.arange(len(comparison_df))
width = 0.35
ax.bar(x - width/2, comparison_df['F1 (with labs)'], width, label='With Labs', color='#2166ac', alpha=0.8)
ax.bar(x + width/2, comparison_df['F1 (without labs)'], width, label='Without Labs', color='#b2182b', alpha=0.8)
ax.set_xticks(x)
ax.set_xticklabels(comparison_df['Model'])
ax.set_ylabel('F1 Macro')
ax.set_title('F1 Score: With vs Without Labs')
ax.legend()
ax.grid(True, alpha=0.3, axis='y')

# Add drop annotations
for i, row in comparison_df.iterrows():
    ax.annotate(f'-{row["F1 Drop %"]:.1f}%', 
                xy=(i + width/2, row['F1 (without labs)'] + 0.02),
                ha='center', fontsize=10, color='red')

# AUC comparison
ax = axes[1]
ax.bar(x - width/2, comparison_df['AUC (with labs)'], width, label='With Labs', color='#2166ac', alpha=0.8)
ax.bar(x + width/2, comparison_df['AUC (without labs)'], width, label='Without Labs', color='#b2182b', alpha=0.8)
ax.set_xticks(x)
ax.set_xticklabels(comparison_df['Model'])
ax.set_ylabel('ROC AUC')
ax.set_title('ROC AUC: With vs Without Labs')
ax.legend()
ax.grid(True, alpha=0.3, axis='y')

plt.suptitle('Impact of Laboratory Features on Model Performance', fontsize=14, fontweight='bold')
plt.tight_layout()
fig.savefig(FIGURES_DIR / 'phase8_labs_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

## 10. Regression Evaluation

In [ ]:
# Generate regression predictions
reg_predictions = {}

# LightGBM with labs
reg_predictions['LightGBM (with labs)'] = lgb_reg_with_labs.predict(X_test_lgb_with_reg.values)

# LightGBM without labs
reg_predictions['LightGBM (without labs)'] = lgb_reg_without_labs.predict(X_test_lgb_without_reg.values)

# MLP with labs
if scaler_reg_with is not None:
    X_scaled = scaler_reg_with.transform(X_test_mlp_with_reg)
else:
    X_scaled = X_test_mlp_with_reg.values
reg_predictions['MLP (with labs)'] = mlp_reg_with_labs.predict(X_scaled)

# MLP without labs
if scaler_reg_without is not None:
    X_scaled = scaler_reg_without.transform(X_test_mlp_without_reg)
else:
    X_scaled = X_test_mlp_without_reg.values
reg_predictions['MLP (without labs)'] = mlp_reg_without_labs.predict(X_scaled)

# Compute metrics
reg_results = {}
for model_name, y_pred in reg_predictions.items():
    reg_results[model_name] = compute_regression_metrics(y_test_reg.values, y_pred)

# Display comparison
reg_df = compare_models_table(
    reg_results,
    metrics=['rmse', 'mae', 'r2', 'mape'],
    sort_by='rmse',
    ascending=True
)

print("\n" + "="*70)
print("REGRESSION MODEL COMPARISON (Test Set) - HbA1c Prediction")
print("="*70)
print(reg_df.to_string())

In [ ]:
# Residual plots for best regression model
best_reg_model = 'LightGBM (with labs)'
y_pred_reg = reg_predictions[best_reg_model]

fig = plot_residuals(y_test_reg.values, y_pred_reg, title=f'{best_reg_model} - Residual Analysis')
fig.savefig(FIGURES_DIR / 'phase8_regression_residuals.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Predicted vs Actual for all regression models
fig, axes = plt.subplots(2, 2, figsize=(12, 10))
axes = axes.flatten()

for i, (model_name, y_pred) in enumerate(reg_predictions.items()):
    ax = axes[i]
    
    ax.scatter(y_pred, y_test_reg.values, alpha=0.3, s=10)
    
    # Perfect prediction line
    min_val = min(y_test_reg.values.min(), y_pred.min())
    max_val = max(y_test_reg.values.max(), y_pred.max())
    ax.plot([min_val, max_val], [min_val, max_val], 'r--', linewidth=2)
    
    # Add metrics
    r2 = reg_results[model_name]['r2']
    rmse = reg_results[model_name]['rmse']
    ax.text(0.05, 0.95, f'R² = {r2:.3f}\nRMSE = {rmse:.3f}',
            transform=ax.transAxes, fontsize=11,
            verticalalignment='top', bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
    
    ax.set_xlabel('Predicted HbA1c (%)')
    ax.set_ylabel('Actual HbA1c (%)')
    ax.set_title(model_name)
    ax.grid(True, alpha=0.3)

plt.suptitle('Predicted vs Actual HbA1c - All Models', fontsize=14, fontweight='bold')
plt.tight_layout()
fig.savefig(FIGURES_DIR / 'phase8_regression_predicted_vs_actual.png', dpi=300, bbox_inches='tight')
plt.show()

---
# Part 2: Model Interpretation (Phase 9)

## 11. SHAP Analysis (LightGBM)

SHAP (SHapley Additive exPlanations) values explain how each feature contributes to the prediction:
- **Positive SHAP value**: Feature pushes prediction toward higher class
- **Negative SHAP value**: Feature pushes prediction toward lower class
- **Magnitude**: Importance of the feature for that prediction

In [ ]:
# Create SHAP explainer for LightGBM
print("Creating SHAP explainer for LightGBM (with labs)...")
print("This uses TreeExplainer which is fast for tree-based models.")

explainer = shap.TreeExplainer(lgb_cls_with_labs)

# Calculate SHAP values for test set
# Using a sample for faster computation (full test set would also work)
n_samples = min(1000, len(X_test_lgb_with))
X_shap = X_test_lgb_with.sample(n=n_samples, random_state=RANDOM_STATE)

# Store feature names before computing SHAP values
feature_names = X_shap.columns.tolist()

print(f"\nComputing SHAP values for {n_samples} samples...")
# Compute SHAP values - pass numpy array
shap_values_raw = explainer.shap_values(X_shap.values)

# Handle different SHAP output formats:
# - Old format (list): shap_values[class_idx] gives shape (n_samples, n_features)
# - New format (3D array): shape (n_samples, n_features, n_classes)
if isinstance(shap_values_raw, list):
    # Old format - list of arrays
    shap_values = shap_values_raw
    print(f"SHAP values format: list of {len(shap_values)} arrays, each {shap_values[0].shape}")
else:
    # New format (SHAP 0.50+) - 3D array (n_samples, n_features, n_classes)
    # Convert to list format for compatibility with our plotting code
    n_classes = shap_values_raw.shape[2]
    shap_values = [shap_values_raw[:, :, i] for i in range(n_classes)]
    print(f"SHAP values format: 3D array {shap_values_raw.shape} -> converted to list")

print(f"Features shape: {X_shap.shape}")
print(f"Number of classes: {len(shap_values)}")

In [ ]:
# SHAP Summary plot (beeswarm) for each class
fig, axes = plt.subplots(1, 3, figsize=(18, 8))

for class_idx, class_name in DIABETES_LABELS.items():
    plt.sca(axes[class_idx])
    shap.summary_plot(
        shap_values[class_idx], 
        X_shap.values,  # Use numpy array
        feature_names=feature_names,  # Explicitly pass feature names
        max_display=15,
        show=False,
        plot_size=None
    )
    axes[class_idx].set_title(f'SHAP Summary: {class_name}', fontsize=12)

plt.suptitle('SHAP Feature Importance by Class', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
fig.savefig(FIGURES_DIR / 'phase9_shap_summary_by_class.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Global feature importance (mean absolute SHAP across all classes)
# Average SHAP values across classes
mean_shap = np.mean([np.abs(sv).mean(axis=0) for sv in shap_values], axis=0)

# Create importance dataframe
importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Mean |SHAP|': mean_shap
}).sort_values('Mean |SHAP|', ascending=False)

# Display top 20
print("\nTop 20 Most Important Features (Mean |SHAP|):")
print("="*50)
print(importance_df.head(20).to_string(index=False))

In [ ]:
# Bar plot of global feature importance
fig, ax = plt.subplots(figsize=(10, 10))

top_n = 20
top_features = importance_df.head(top_n)

y_pos = np.arange(top_n)
ax.barh(y_pos, top_features['Mean |SHAP|'], color='steelblue', alpha=0.8)
ax.set_yticks(y_pos)
ax.set_yticklabels(top_features['Feature'])
ax.invert_yaxis()  # Top feature at top
ax.set_xlabel('Mean |SHAP Value|')
ax.set_title('Top 20 Features by SHAP Importance', fontsize=14, fontweight='bold')
ax.grid(True, alpha=0.3, axis='x')

plt.tight_layout()
fig.savefig(FIGURES_DIR / 'phase9_shap_importance_bar.png', dpi=300, bbox_inches='tight')
plt.show()

## 12. SHAP Dependence Plots

In [ ]:
# SHAP dependence plots for top features (for Diabetes class)
diabetes_class_idx = 2  # Diabetes
top_features_list = importance_df.head(6)['Feature'].tolist()

fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

for i, feature in enumerate(top_features_list):
    ax = axes[i]
    # Get feature index
    feature_idx = feature_names.index(feature)
    shap.dependence_plot(
        feature_idx,  # Use index instead of name
        shap_values[diabetes_class_idx],
        X_shap.values,
        feature_names=feature_names,
        ax=ax,
        show=False
    )
    ax.set_title(f'{feature}', fontsize=11)

plt.suptitle('SHAP Dependence Plots - Top Features (Diabetes Class)', fontsize=14, fontweight='bold')
plt.tight_layout()
fig.savefig(FIGURES_DIR / 'phase9_shap_dependence.png', dpi=300, bbox_inches='tight')
plt.show()

## 13. Permutation Importance

In [ ]:
# Permutation importance (model-agnostic)
print("Computing permutation importance (this may take a minute)...")

# Use a subset for speed
n_perm = min(500, len(X_test_lgb_with))
X_perm = X_test_lgb_with.sample(n=n_perm, random_state=RANDOM_STATE)
y_perm = y_test_cls.loc[X_perm.index]

perm_importance = permutation_importance(
    lgb_cls_with_labs,
    X_perm.values,  # Use numpy array
    y_perm.values,  # Use numpy array
    n_repeats=10,
    random_state=RANDOM_STATE,
    scoring='f1_macro',
    n_jobs=-1
)

perm_df = pd.DataFrame({
    'Feature': X_perm.columns,
    'Importance': perm_importance.importances_mean,
    'Std': perm_importance.importances_std
}).sort_values('Importance', ascending=False)

print("\nTop 20 Features by Permutation Importance:")
print("="*50)
print(perm_df.head(20).to_string(index=False))

In [ ]:
# Compare SHAP vs Permutation importance
fig, axes = plt.subplots(1, 2, figsize=(14, 8))

top_n = 15

# SHAP
ax = axes[0]
top_shap = importance_df.head(top_n)
y_pos = np.arange(top_n)
ax.barh(y_pos, top_shap['Mean |SHAP|'], color='steelblue', alpha=0.8)
ax.set_yticks(y_pos)
ax.set_yticklabels(top_shap['Feature'])
ax.invert_yaxis()
ax.set_xlabel('Mean |SHAP Value|')
ax.set_title('SHAP Importance', fontsize=12)
ax.grid(True, alpha=0.3, axis='x')

# Permutation
ax = axes[1]
top_perm = perm_df.head(top_n)
ax.barh(y_pos, top_perm['Importance'], xerr=top_perm['Std'], color='coral', alpha=0.8)
ax.set_yticks(y_pos)
ax.set_yticklabels(top_perm['Feature'])
ax.invert_yaxis()
ax.set_xlabel('Mean F1 Decrease')
ax.set_title('Permutation Importance', fontsize=12)
ax.grid(True, alpha=0.3, axis='x')

plt.suptitle('Feature Importance: SHAP vs Permutation', fontsize=14, fontweight='bold')
plt.tight_layout()
fig.savefig(FIGURES_DIR / 'phase9_importance_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

## 14. Modifiable vs Non-Modifiable Risk Factors

In [ ]:
# Categorize features as modifiable vs non-modifiable
modifiable_patterns = [
    'BMX',  # Body measures (weight, BMI, waist)
    'DR1',  # Dietary intake
    'PAQ', 'PAD',  # Physical activity
    'SL',  # Sleep
    'ALQ',  # Alcohol
    'SMQ', 'SMD',  # Smoking
    'BPX',  # Blood pressure (partially modifiable)
    'WEIGHT',  # Derived weight features
    'WAIST',  # Waist-height ratio
    'PHQ9',  # Depression (treatable)
    'TOTAL_WATER',
    'CARB', 'SUGAR', 'SAT_FAT',  # Dietary derived
    'AVG_SYS', 'AVG_DIA', 'PULSE', 'MAP', 'BP_',  # BP derived
]

non_modifiable_patterns = [
    'RIDAGEYR',  # Age
    'RIAGENDR',  # Gender
    'MCQ300C',  # Family history
    'MCQ160',  # Medical history (CHF, CHD, etc.)
    'ANY_CVD',  # CVD history
]

def categorize_feature(feature):
    for pattern in non_modifiable_patterns:
        if pattern in feature:
            return 'Non-Modifiable'
    for pattern in modifiable_patterns:
        if pattern in feature:
            return 'Modifiable'
    # Lab values are not directly modifiable (but reflect modifiable factors)
    if feature.startswith('LB') or feature.startswith('UR') or 'ACR' in feature or 'TG_HDL' in feature:
        return 'Lab Value'
    return 'Other'

importance_df['Category'] = importance_df['Feature'].apply(categorize_feature)

# Summary by category
category_summary = importance_df.groupby('Category')['Mean |SHAP|'].agg(['sum', 'mean', 'count'])
category_summary.columns = ['Total Importance', 'Mean Importance', 'N Features']
category_summary = category_summary.sort_values('Total Importance', ascending=False)

print("\nFeature Importance by Category:")
print("="*60)
print(category_summary.round(4).to_string())

In [ ]:
# Top modifiable features
print("\nTop 15 Modifiable Risk Factors:")
print("="*50)
modifiable_df = importance_df[importance_df['Category'] == 'Modifiable'].head(15)
print(modifiable_df[['Feature', 'Mean |SHAP|']].to_string(index=False))

In [ ]:
# Visualize modifiable vs non-modifiable
fig, axes = plt.subplots(1, 2, figsize=(14, 8))

# Pie chart of total importance by category
ax = axes[0]
colors = {'Modifiable': '#2ca02c', 'Non-Modifiable': '#d62728', 'Lab Value': '#1f77b4', 'Other': '#7f7f7f'}
cat_colors = [colors.get(cat, '#7f7f7f') for cat in category_summary.index]
ax.pie(category_summary['Total Importance'], labels=category_summary.index, autopct='%1.1f%%',
       colors=cat_colors, startangle=90)
ax.set_title('Share of Total SHAP Importance', fontsize=12)

# Bar chart of top features colored by category
ax = axes[1]
top_20 = importance_df.head(20)
y_pos = np.arange(20)
bar_colors = [colors.get(cat, '#7f7f7f') for cat in top_20['Category']]
ax.barh(y_pos, top_20['Mean |SHAP|'], color=bar_colors, alpha=0.8)
ax.set_yticks(y_pos)
ax.set_yticklabels(top_20['Feature'])
ax.invert_yaxis()
ax.set_xlabel('Mean |SHAP Value|')
ax.set_title('Top 20 Features (Colored by Category)', fontsize=12)
ax.grid(True, alpha=0.3, axis='x')

# Add legend
from matplotlib.patches import Patch
legend_elements = [Patch(facecolor=colors[k], label=k) for k in colors]
ax.legend(handles=legend_elements, loc='lower right')

plt.suptitle('Modifiable vs Non-Modifiable Risk Factors', fontsize=14, fontweight='bold')
plt.tight_layout()
fig.savefig(FIGURES_DIR / 'phase9_modifiable_factors.png', dpi=300, bbox_inches='tight')
plt.show()

## 15. Actionable Insights Summary

In [ ]:
# Generate actionable insights
print("\n" + "="*70)
print("ACTIONABLE INSIGHTS FOR DIABETES PREVENTION")
print("="*70)

print("\n📊 MODEL PERFORMANCE SUMMARY")
print("-" * 40)
best_model = 'LightGBM (with labs)'
print(f"Best Model: {best_model}")
print(f"  - F1 Macro: {results[best_model]['f1_macro']:.3f}")
print(f"  - ROC AUC: {results[best_model]['roc_auc_ovr']:.3f}")
print(f"  - Accuracy: {results[best_model]['accuracy']:.3f}")

print("\n🎯 TOP MODIFIABLE RISK FACTORS")
print("-" * 40)
for i, (_, row) in enumerate(modifiable_df.head(10).iterrows(), 1):
    print(f"{i:2}. {row['Feature']}: {row['Mean |SHAP|']:.4f}")

print("\n💡 KEY RECOMMENDATIONS")
print("-" * 40)
recommendations = [
    "1. WEIGHT MANAGEMENT: BMI and waist circumference are top predictors."
    "   Maintaining healthy weight is the most impactful modifiable factor.",
    "2. PHYSICAL ACTIVITY: Regular exercise reduces diabetes risk."
    "   Both vigorous and moderate activity contribute.",
    "3. DIETARY QUALITY: Monitor carbohydrate quality (fiber ratio),"
    "   saturated fat intake, and sugar consumption.",
    "4. BLOOD PRESSURE: Hypertension is strongly associated with diabetes."
    "   Regular monitoring and management is important.",
    "5. SLEEP: Sleep duration and quality affect metabolic health."
    "   Aim for consistent 7-8 hours per night.",
]
for rec in recommendations:
    print(rec)
    print()

print("\n⚠️ LIMITATIONS")
print("-" * 40)
print("- Cross-sectional data: Cannot establish causation")
print("- Prediabetes is hardest to predict (F1 ~0.56)")
print("- Lab values provide ~10% F1 improvement")
print("- Model performs slightly worse for older age groups")

## 16. Save Results

In [ ]:
# Save comprehensive results
evaluation_results = {
    'classification': {
        model: {k: float(v) if isinstance(v, (np.floating, float)) else v 
                for k, v in metrics.items() if k != 'confusion_matrix'}
        for model, metrics in results.items()
    },
    'regression': {
        model: {k: float(v) for k, v in metrics.items()}
        for model, metrics in reg_results.items()
    },
    'feature_importance': {
        'shap_top_20': importance_df.head(20).to_dict('records'),
        'permutation_top_20': perm_df.head(20).to_dict('records'),
        'importance_by_category': category_summary.to_dict(),
    },
    'subgroup_analysis': {
        'by_age': age_results.to_dict('records') if len(age_results) > 0 else [],
        'by_gender': gender_results.to_dict('records') if len(gender_results) > 0 else [],
        'by_bmi': bmi_results.to_dict('records') if len(bmi_results) > 0 else [],
        'by_race': race_results.to_dict('records') if len(race_results) > 0 else [],
    },
    'labs_comparison': comparison_df.to_dict('records'),
}

# Save to JSON
results_path = MODELS_DIR / 'evaluation_results.json'
with open(results_path, 'w') as f:
    json.dump(evaluation_results, f, indent=2, default=str)

print(f"Results saved to: {results_path}")

In [ ]:
# Save feature importance to CSV for easy reference
importance_df.to_csv(MODELS_DIR / 'feature_importance_shap.csv', index=False)
perm_df.to_csv(MODELS_DIR / 'feature_importance_permutation.csv', index=False)

print(f"Feature importance saved to:")
print(f"  - {MODELS_DIR / 'feature_importance_shap.csv'}")
print(f"  - {MODELS_DIR / 'feature_importance_permutation.csv'}")

## 17. Summary

In [ ]:
print("\n" + "="*70)
print("PHASE 8 & 9 SUMMARY")
print("="*70)

print("\n📈 FIGURES GENERATED:")
figures = list(FIGURES_DIR.glob('phase8_*.png')) + list(FIGURES_DIR.glob('phase9_*.png'))
for fig_path in sorted(figures):
    print(f"  - {fig_path.name}")

print(f"\n📁 ARTIFACTS SAVED:")
print(f"  - {MODELS_DIR / 'evaluation_results.json'}")
print(f"  - {MODELS_DIR / 'feature_importance_shap.csv'}")
print(f"  - {MODELS_DIR / 'feature_importance_permutation.csv'}")

print("\n✅ KEY FINDINGS:")
print(f"  1. Best classification model: LightGBM (with labs) - F1={results['LightGBM (with labs)']['f1_macro']:.3f}")
print(f"  2. Best regression model: LightGBM (with labs) - R²={reg_results['LightGBM (with labs)']['r2']:.3f}")
print(f"  3. Labs provide ~{comparison_df.iloc[0]['F1 Drop %']:.1f}% F1 improvement for LightGBM")
print(f"  4. Top modifiable factors: BMI, waist, dietary patterns, physical activity")
print(f"  5. Prediabetes is hardest to predict (lowest per-class F1)")

---
## Next Steps

- **Phase 10**: Deployment (Streamlit app, API)
- **Phase 11**: Documentation & Polish (README, final report)